# Sequence Alignment

**Code below shouldn't be modified.**

It implements a global sequence alignment algorithm.

In [46]:
import pandas as pd
import numpy as np

In [47]:
def read_fasta(file):
    seqs = []
    headers = []
    # implement a read fasta file
    # YOUR SOLUTION HERE
    ## BEGIN SOLUTION
    f = open(file)
    header = None
    sequence = []
    for line in f:
        line = line.strip()
        if line[0] == ">":
            if header is not None:
                headers.append(header)
                seq = "".join(sequence)
                seqs.append(seq.strip())
                sequence = []
            header = line
        else:
            sequence.append(line.strip())
    if header is not None:
        headers.append(header)
        seq = "".join(sequence)
        seqs.append(seq.strip())
        sequence = []
    ## END SOLUTION    
    return headers,seqs

In [48]:
def align_dynamic2(s1,s2,match_score=1,mismatch_score=0,gap_score=0,verbose=False):
    scores = pd.DataFrame(index=["-"]+[s1[:i+1] for i in range(len(s1))],columns=["-"]+[s2[:i+1] for i in range(len(s2))])
    aligned = pd.DataFrame(index=["-"]+[s1[:i+1] for i in range(len(s1))],columns=["-"]+[s2[:i+1] for i in range(len(s2))])
    for s2_part in scores.columns:
        scores.loc["-",s2_part] = 0
        if s2_part == "-":
            aligned.loc["-","-"] = ("","")
        else:
            aligned.loc["-",s2_part] = ("".join(["-" for i in range(len(s2_part))]),s2_part)
    for s1_part in scores.index:
        scores.loc[s1_part,"-"] = 0
        if s1_part == "-":
            aligned.loc["-","-"] = ("","")
        else:
            aligned.loc[s1_part,"-"] = (s1_part,"".join(["-" for i in range(len(s1_part))]))
    if verbose:
        display(aligned)
    
    nrows,ncols = scores.shape
    for i in range(1,nrows):
        for j in range(1,ncols):
            # What are our three options
            opt1_s1 = scores.index[i-1] # remember the rows are representative of s1
            opt1_s2 = scores.columns[j-1] # remember the columns are representative of s2
            score_opt1 = -np.Inf # FIX THIS!
            s1_aligned_opt1 = "" # FIX THIS!
            s2_aligned_opt1 = "" # FIX THIS!
            ## BEGIN SOLUTION
            if scores.index[i][-1]==scores.columns[j][-1]: 
                score = match_score
            else:
                score = mismatch_score
            score_opt1 = scores.loc[opt1_s1,opt1_s2] + score
            s1_aligned_opt1 = aligned.loc[opt1_s1,opt1_s2][0] + scores.index[i][-1]
            s2_aligned_opt1 = aligned.loc[opt1_s1,opt1_s2][1] + scores.columns[j][-1]
            ## END SOLUTION
            
            opt2_s1 = scores.index[i-1]
            opt2_s2 = scores.columns[j]
            score_opt2 = -np.Inf # FIT THIS!
            s1_aligned_opt2 = "" # FIX THIS!
            s2_aligned_opt2 = "" # FIX THIS!
            ## BEGIN SOLUTION
            score_opt2 = scores.loc[opt2_s1,opt2_s2]+gap_score
            s1_aligned_opt2 = aligned.loc[opt2_s1,opt2_s2][0] + scores.index[i][-1]
            s2_aligned_opt2 = aligned.loc[opt2_s1,opt2_s2][1] + "-"
            ## END SOLUTION
            
            opt3_s1 = scores.index[i]
            opt3_s2 = scores.columns[j-1]
            score_opt3 = -np.Inf # FIT THIS!
            s1_aligned_opt3 = "" # FIX THIS!
            s2_aligned_opt3 = "" # FIX THIS!
            ## BEGIN SOLUTION
            score_opt3 = scores.loc[opt3_s1,opt3_s2]+gap_score
            s1_aligned_opt3 = aligned.loc[opt3_s1,opt3_s2][0] + "-"
            s2_aligned_opt3 = aligned.loc[opt3_s1,opt3_s2][1] + scores.columns[j][-1]
            ## END SOLUTION
            
            scores.loc[scores.index[i],scores.columns[j]] = max(score_opt1,score_opt2,score_opt3)
            if max(score_opt1,score_opt2,score_opt3) == score_opt1:
                aligned.loc[scores.index[i],scores.columns[j]] = (s1_aligned_opt1,s2_aligned_opt1)
            elif max(score_opt1,score_opt2,score_opt3) == score_opt2:
                aligned.loc[scores.index[i],scores.columns[j]] = (s1_aligned_opt2,s2_aligned_opt2)
            else:
                aligned.loc[scores.index[i],scores.columns[j]] = (s1_aligned_opt3,s2_aligned_opt3)
    if verbose:
        display(scores)
        display(aligned)
    return scores.loc[s1,s2],aligned.loc[s1,s2][0],aligned.loc[s1,s2][1]

In [49]:
def print_alignment(aligned_s1,aligned_s2,num_to_print=100):
    num_to_print=100
    for i in range(num_to_print):
        print(aligned_s1[i],end="")
    print()
    for i in range(num_to_print):
        if aligned_s1[i] == aligned_s2[i]:
            print("|",end="")
        else:
            print(" ",end="")
    print()
    for i in range(num_to_print):
        print(aligned_s2[i],end="")

### Feel free to change the arguments/numbers below and rerun as many times as you would like

In [42]:
s1="CGCAACCACAGCGCGCAGGGCAGGCGCGAGCTGTCTGAGCCCCGGCCTCGGACCGCCCACTGGACTCCCGGCACGCCCGGTGCCGCCTTCCGGCTCCAGTCCCCCGGGCTCGGCCTCGGCGAGGTGTAATTCGCTTGCGCGGGCCGGCCCCGGAGGCTCTCGGCGAGCGCGGCGCGGTA"
s2="CGCAACGGCAGCGCGCAGGGCAGGCGCGAGCTGGCCTCTGAGCCCCGGCCTCGGACCGCCCACTCCACGCCCGGCAGGCCCGGTGCCGCCTTCCGGCTCCAGTCCCCCCGCTCGGCCTCGGCGAGGTGTAATTCGCAGGCGGGCCGGCTCCGGAGGCTCTCAACGGCGAGCGCGGTGCGGTA"
score_1,aligned_s1_1,aligned_s2_1 = align_dynamic2(s1,s2,match_score=1,mismatch_score=0,gap_score=0)
print(score_1)

166


In [50]:
print_alignment(aligned_s1_1,aligned_s2_1)

CGCAACCACAGCGCGCAGGGCAGGCGCGAGCT-G--TCTGAGCCCCGGCCTCGGACCGCCCACTGGACTCCCGGCACGCCCGGTGCCGCCTTCCGGCTCC
||||||  |||||||||||||||||||||||| |  ||||||||||||||||||||||||||||  || ||||||| |||||||||||||||||||||||
CGCAACGGCAGCGCGCAGGGCAGGCGCGAGCTGGCCTCTGAGCCCCGGCCTCGGACCGCCCACTCCACGCCCGGCAGGCCCGGTGCCGCCTTCCGGCTCC

In [55]:
score_2,aligned_s1_2,aligned_s2_2 = align_dynamic2(s1,s2,match_score=2,mismatch_score=-3,gap_score=-1)
print(score_2)

303


In [56]:
print_alignment(aligned_s1_2,aligned_s2_2)

CGCAAC--CACAGCGCGCAGGGCAGGCGCGAGCT-G--TCTGAGCCCCGGCCTCGGACCGCCCACT--GGAC-TCCCGGCA-CGCCCGGTGCCGCCTTCC
||||||    |||||||||||||||||||||||| |  ||||||||||||||||||||||||||||    ||  |||||||  |||||||||||||||||
CGCAACGG--CAGCGCGCAGGGCAGGCGCGAGCTGGCCTCTGAGCCCCGGCCTCGGACCGCCCACTCC--ACG-CCCGGCAG-GCCCGGTGCCGCCTTCC